In [1]:
import pandas as pd
import sys
sys.path.append('../')
from evaluation.generated_dataset import GeneratedDataset

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/matminer/utils/data.py:326: UserWarning: MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.
  warnings.warn(f"{self.__class__.__name__}(impute_nan=False):\n" + IMPUTE_NAN_WARNING)


In [2]:
mp_20_source_sample = GeneratedDataset.from_transformations(("split", "test", "1k-sample")).data

In [3]:
# Sample is due to a bug when generating the dataset
permuted_sample = mp_20_source_sample.sample(1000, random_state=42).reset_index(drop=True)
permuted_to_initial_index = pd.Series(mp_20_source_sample.index, index=permuted_sample.index)
diffcsp_sample = GeneratedDataset.from_cache(("split", "test", "1k-sample", "DiffCSP++")).data
#diffcsp_sample.index = permuted_sample.index.map(permuted_to_initial_index)

In [4]:
matching_sg = (diffcsp_sample.spacegroup_number == permuted_sample.spacegroup_number)
print(f"Matching spacegroup numbers: {matching_sg.sum()} out of {len(matching_sg)}")

Matching spacegroup numbers: 45 out of 1000


In [5]:
from pymatgen.analysis.structure_matcher import StructureMatcher
sm = StructureMatcher()
structure_matches = pd.Series(
    [sm.fit(s1, s2) for s1, s2 in zip(diffcsp_sample.structure, mp_20_source_sample.structure)],
    index=diffcsp_sample.index
)

print(f"Matching structures: {structure_matches.sum()} out of {len(structure_matches)}")

Matching structures: 1 out of 1000
